In [1]:
# Add scripts directory to path
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'scripts'))
import pytz

# Import geometry functions
from geometry import (
    download_country_shapes,
    download_nuts3_shapes,
    join_shapes,
    point_in_shape,
    mask_shape,
    buffer_shape,
    get_shape_area,
    get_european_union_shape,
    load_shapes_efficiently,
    to_rect,
    to_point
)

# For visualization
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from scipy.optimize import curve_fit

print("✓ Setup complete")

# default figures path

figures_path = Path("/Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model/results/figures")

def find_repo_root(max_up=6):
    p = Path.cwd().resolve()
    for _ in range(max_up):
        if (p / 'README.md').exists() or (p / '.git').exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    return Path.cwd().resolve()

repo_root = find_repo_root()

print(f"Repository root: {repo_root}")


✓ Setup complete
Repository root: /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model


In [2]:
import json

JOIN = False

# Define cache dir
cache_dir = Path.cwd().parent / 'data' / 'cache' / 'geometry'

# Candidate paths (in preferred order)

all_countries_candidates = [
    cache_dir / 'all_countries.geojson',
    cache_dir / 'all_countries.parquet',
]
if JOIN:
    voronoi_candidates = [
        cache_dir / 'voronoi_eu27_join_P.geojson',
        cache_dir / 'voronoi_eu27_join_P.parquet',
        cache_dir / 'voronoi_eu27_join.geojson',
        cache_dir / 'voronoi_eu27_join.parquet',
    ]
    csv_candidates = [
        cache_dir / 'voronoi_eu27_join_population.csv',
    ]
else:
    voronoi_candidates = [
        cache_dir / 'voronoi_eu27_P.geojson',
        cache_dir / 'voronoi_eu27_P.parquet',
        cache_dir / 'voronoi_eu27.geojson',
        cache_dir / 'voronoi_eu27.parquet',
    ]
    csv_candidates = [
        cache_dir / 'voronoi_eu27_population.csv',
    ]

def first_existing(paths):
    for p in paths:
        if p.exists():
            return p
    return None

all_countries_path = first_existing(all_countries_candidates)
voronoi_eu27_path = first_existing(voronoi_candidates)

if all_countries_path and voronoi_eu27_path:
    # Load geometries
    all_countries_geojson = load_shapes_efficiently(all_countries_path)
    try: 
        all_countries_gdf = gpd.GeoDataFrame.from_features(all_countries_geojson['features'])
    except Exception as e:
        print(f"⚠ Failed to load all_countries GeoDataFrame: {e}")
        all_countries_gdf = all_countries_geojson
    
    voronoi_geojson = load_shapes_efficiently(voronoi_eu27_path)
    try:
        voronoi_gdf = gpd.GeoDataFrame.from_features(voronoi_geojson['features'])
    except Exception as e:
        print(f"⚠ Failed to load voronoi_eu27 GeoDataFrame: {e}")
        voronoi_gdf = voronoi_geojson
    
    # Load buses
    buses_path = Path.cwd().parent / 'data' / 'raw' / 'OSM Prebuilt Electricity Network' / 'buses.csv'
    if buses_path.exists():
        buses_data = pd.read_csv(buses_path)
        print(f"✓ Loaded {len(all_countries_gdf)} countries")
        print(f"✓ Loaded {len(voronoi_gdf)} Voronoi cells")
        print(f"✓ Loaded {len(buses_data)} buses")
    else:
        buses_data = None
        print(f"⚠ Buses file not found at {buses_path}")
else:
    print("⚠ Cache files not found")
    print("  Tried all_countries candidates:")
    for p in all_countries_candidates:
        print(f"    - {p} (exists={p.exists()})")
    print("  Tried voronoi candidates:")
    for p in voronoi_candidates:
        print(f"    - {p} (exists={p.exists()})")
    buses_data = None


⚠ Failed to load all_countries GeoDataFrame: 'features'
⚠ Failed to load voronoi_eu27 GeoDataFrame: 'features'
✓ Loaded 35 countries
✓ Loaded 6258 Voronoi cells
✓ Loaded 6737 buses


In [3]:
# Load csv candidates
pop_csv = first_existing(csv_candidates)

# To pandasa DataFrame
if pop_csv:
    population_data = pd.read_csv(pop_csv, index_col=0)
    print(f"\n✓ Loaded population data from {pop_csv} with {len(population_data)} entries.")
    print("\nSample population data:")
    display(population_data.head())


✓ Loaded population data from /Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/European Energy Policy/HA/PyPSA---Simplified-European-Model/PyPSA---Simplified-European-Model/data/cache/geometry/voronoi_eu27_population.csv with 6258 entries.

Sample population data:


,population,EU_population_share
bus_id,,
AT1-220,9941.0,0.000022
AT10-220,43914.0,0.000097
AT11-220,49063.0,0.000109
AT12-380,58700.0,0.000130
AT13-380,84807.0,0.000188


In [5]:
load_data_path = repo_root / "data" / "raw" / "energy_charts" / "processed_load_data.csv"
load_data = pd.read_csv(load_data_path, index_col=0, parse_dates=True, dtype={'Country': str, 'Load': float, 'Missing': str})





In [6]:
JOIN = True
voronoi_demand = load_shapes_efficiently(repo_root / 'data' / 'processed' / f"voronoi_demand{'_join' if JOIN else ''}.csv")

In [7]:
voronoi_demand

,bus_id,bus_x,bus_y,region,country,area_km2,bounded,population,EU_population_share,demand_series,geometry
0,AT1-220,16.807409,48.709909,combined,AT,1385.430479,True,139398.0,0.000308,Timestamp\n2015-01-01 00:00:00+00:00 1.8556...,"POLYGON ((16.47213 48.68105, 16.44706 48.7116,..."
1,AT10-220,13.212633,47.330377,combined,AT,998.596542,True,43914.0,0.000097,Timestamp\n2015-01-01 00:00:00+00:00 0.5845...,"POLYGON ((12.9523 47.51901, 13.15077 47.59155,..."
2,AT11-220,11.102718,47.293961,combined,AT,531.999239,True,56400.0,0.000125,Timestamp\n2015-01-01 00:00:00+00:00 0.7507...,"POLYGON ((10.98191 47.57542, 11.26117 47.3334,..."
3,AT12-380,16.166650,47.270734,combined,AT,654.566186,True,58819.0,0.000130,Timestamp\n2015-01-01 00:00:00+00:00 0.7829...,"POLYGON ((16.26957 47.52466, 16.14328 46.83168..."
4,AT13-380,15.990006,47.257024,combined,AT,1188.430975,True,84807.0,0.000188,Timestamp\n2015-01-01 00:00:00+00:00 1.1289...,"POLYGON ((16.06185 47.47621, 16.10692 46.89546..."
...,...,...,...,...,...,...,...,...,...,...,...
6253,way/666404730-400,20.534095,48.964547,combined,SK,3764.990829,True,405038.0,0.000896,Timestamp\n2015-01-01 00:00:00+00:00 2.5627...,"POLYGON ((20.74196 48.71628, 20.67532 48.62227..."
6254,way/672771472-400,18.863051,49.213694,combined,SK,1522.932044,True,271166.0,0.000600,Timestamp\n2015-01-01 00:00:00+00:00 1.7157...,"POLYGON ((18.69294 48.94571, 18.59313 49.39031..."
6255,way/783483874-220,21.976714,48.555380,combined,SK,100.640286,True,10213.0,0.000023,Timestamp\n2015-01-01 00:00:00+00:00 0.0646...,"POLYGON ((22.03629 48.43075, 22.04621 48.38432..."
6256,way/783483874-400,21.977044,48.555537,combined,SK,53.619192,True,4212.0,0.000009,Timestamp\n2015-01-01 00:00:00+00:00 0.0266...,"POLYGON ((21.98433 48.67562, 22.03629 48.43075..."


In [14]:
load_data[load_data['Country'] == 'IE']

,Country,Load,Missing
Timestamp,,,
2015-01-01 00:00:00+00:00,IE,2903.3,False
2015-01-01 01:00:00+00:00,IE,2726.2,False
2015-01-01 02:00:00+00:00,IE,2487.6,False
2015-01-01 03:00:00+00:00,IE,2311.8,False
2015-01-01 04:00:00+00:00,IE,2166.5,False
...,...,...,...
2024-12-31 19:00:00+00:00,IE,4353.4,NaN
2024-12-31 20:00:00+00:00,IE,4079.9,NaN
2024-12-31 21:00:00+00:00,IE,3837.0,NaN


In [9]:
x = voronoi_demand.loc[0,'demand_series']
display(x)

'Timestamp\n2015-01-01 00:00:00+00:00    1.855651\n2015-01-01 01:00:00+00:00    1.773496\n2015-01-01 02:00:00+00:00    1.667533\n2015-01-01 03:00:00+00:00    1.629786\n2015-01-01 04:00:00+00:00    1.628429\n                               ...   \n2024-12-31 19:00:00+00:00    2.207063\n2024-12-31 20:00:00+00:00    2.098694\n2024-12-31 21:00:00+00:00    2.098571\n2024-12-31 22:00:00+00:00    2.021967\n2024-12-31 23:00:00+00:00    2.021967\nLength: 87672, dtype: float64'

In [12]:
x.split()

['Timestamp',
 '2015-01-01',
 '00:00:00+00:00',
 '1.855651',
 '2015-01-01',
 '01:00:00+00:00',
 '1.773496',
 '2015-01-01',
 '02:00:00+00:00',
 '1.667533',
 '2015-01-01',
 '03:00:00+00:00',
 '1.629786',
 '2015-01-01',
 '04:00:00+00:00',
 '1.628429',
 '...',
 '2024-12-31',
 '19:00:00+00:00',
 '2.207063',
 '2024-12-31',
 '20:00:00+00:00',
 '2.098694',
 '2024-12-31',
 '21:00:00+00:00',
 '2.098571',
 '2024-12-31',
 '22:00:00+00:00',
 '2.021967',
 '2024-12-31',
 '23:00:00+00:00',
 '2.021967',
 'Length:',
 '87672,',
 'dtype:',
 'float64']